In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2019-01-17 11:11:06


In [7]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 15

Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool'
Project_GDB  = Project_DIR + '\\2.Project_GDB.gdb'
ContractsDir = Project_DIR + '\\4.Contracts_AnalysisFiles'
DocumentsDir = Project_DIR + '\\5.Documents'
PythonDir    = Project_DIR + '\\6.Python_Codes'
ReceivedDir  = Project_DIR + '\\7.ReceivedData'
XML_Dir      = ReceivedDir + '\\HSIP Candidate Applications'


# IRIS Dataset
IRISPath = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\5.0_Roadway_Layer\IRIS'
IRISPath = r'C:\Users\mr068144\Downloads\IL Data\IRIS\IL_IRIS.gdb'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route') for y in range(2005,2018)}
IRIS_Tab   = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table_HCurve') for y in Years}
IRIS_Tab[2012] = IRIS_Tab[2013]

# Intersections Dataset
IntPG = r'C:\Users\mr068144\Downloads\6.0_Intersection_Layer\IRIS_Ints.gdb\Int2014_PG'
Intersections = {year:IntPG for year in Years}

#Crash Dataset
CrashData  = {y:os.path.abspath(r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data\IL_Crash.gdb\\' +
                          'CrashExtract_' + str(y) + '_GIS') for y in Years}

Fields = ['AADT',
          'O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH',
          'I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH',
          'LNS','LN_WTH','MED_TYP','MED_WTH','SURF_WTH','URBAN','PG','Radius','CurveLen']
#Fields = ['AADT','URBAN','PG']
RouteID = 'INVENTORY'
BMP = 'BEG_STA'
EMP = 'END_STA'

# Geocoded projects
HSIP_Int  = Project_GDB + '\\WPPS_Int'
HSIP_Seg  = Project_GDB + '\\WPPS_Seg'

HSIP_Path    = ReceivedDir + '\\HSIP_AllApprovedProjects.xlsm'
WPPS_Ob_Path = ReceivedDir + '\\HSIP Contract and Obligation Data.xlsx'
WPPS_Pr_Path = ReceivedDir + '\\HSIP Project Programmed Data.xlsx'
SPF_Data     = ReceivedDir  + '\\SPF Coefficients.xlsx'

2019-01-17 11:20:36


In [4]:
XML_DF = pd.read_excel(DocumentsDir+'\\XML_DF.xlsx')
XML_DF.index = XML_DF.HSIPID.astype(int)
Cont_DF = pd.read_excel(DocumentsDir+'\\Cont_DF.xlsx',index_col=0)
HSIP_DF = pd.read_excel(DocumentsDir+'\\HSIP_DF.xlsx')
HSIP_DF.index = HSIP_DF['HSIP ID'].astype(int)
HSIP_Joined = pd.read_excel(DocumentsDir+'\\HSIP_Joined.xlsx')
HSIP_Joined.index = HSIP_Joined.HSIPID.astype(int)

In [3]:
WPPS_Int = hsmpy3.common.FCtoDF(Project_GDB + '\\WPPS_Int',readGeometry=True)
WPPS_Int['X'] = [s.firstPoint.X for s in WPPS_Int.Shape]
WPPS_Int['Y'] = [s.firstPoint.Y for s in WPPS_Int.Shape]
WPPS_Seg = hsmpy3.common.FCtoDF(Project_GDB + '\\WPPS_Seg')
print(WPPS_Int.shape,WPPS_Seg.shape)
Const_DC = pd.read_excel(Project_DIR+'\\11.Plans_DataCollection\\PlansDataCollection_12062018.xlsx',sheetname='Sheet1',index_col=[0,1])
Const_Seg = pd.read_excel(Project_DIR+'\\11.Plans_DataCollection\\PlansDataCollection_12062018.xlsx',sheetname='Segments')
Const_Int = pd.read_excel(Project_DIR+'\\11.Plans_DataCollection\\PlansDataCollection_12062018.xlsx',sheetname='Intersections')
print(Const_DC.shape,Const_Seg.shape,Const_Int.shape)
# OK Contracts
ok_cont = list(Const_DC.loc[Const_DC.Location=='OK'].index.droplevel(1))
Const_Int = Const_Int[~Const_Int.ContNum.isin(ok_cont)]
Const_Seg = Const_Seg[~Const_Seg.ContNum.isin(ok_cont)]
print(Const_DC.shape,Const_Seg.shape,Const_Int.shape)


(341, 6) (1043, 7)
(401, 18) (126, 5) (76, 3)
(401, 18) (124, 5) (76, 3)


In [5]:
#hsmpy3.common.ListFCinGDBorMDB(r'C:\Users\mr068144\Downloads\IL Data\6.0_Intersection_Layer\IRIS_Ints.gdb')
#hsmpy3.common.PrintSummary(r'C:\Users\mr068144\Downloads\IL Data\6.0_Intersection_Layer\IRIS_Ints.gdb\Int2014_PG')
Int_DF = hsmpy3.common.FCtoDF(r'C:\Users\mr068144\Downloads\IL Data\6.0_Intersection_Layer\IRIS_Ints.gdb\Int2014_PG',readGeometry=True,selectedFields=['OBJECTID'])
Int_DF['X'] = [s.firstPoint.X for s in Int_DF.Shape]
Int_DF['Y'] = [s.firstPoint.Y for s in Int_DF.Shape]

In [6]:
Const_Loc = pd.DataFrame(columns=['ContNum','X','Y','INVENTORY','BMP','EMP'])
j = 0
for i,r in WPPS_Int[WPPS_Int.ContNum.isin(ok_cont)].iterrows():
    Const_Loc.loc[j,'ContNum'] = r.ContNum
    Const_Loc.loc[j,'X'] = r.X
    Const_Loc.loc[j,'Y'] = r.Y
    j += 1
for i,r in WPPS_Seg[WPPS_Seg.ContNum.isin(ok_cont)].iterrows():
    Const_Loc.loc[j,'ContNum'] = r.ContNum
    Const_Loc.loc[j,'INVENTORY'] = r.INVENTORY
    Const_Loc.loc[j,'BMP'] = r.BEG_STA
    Const_Loc.loc[j,'EMP'] = r.END_STA
    j += 1
for i,r in Const_Int[~pd.isnull(Const_Int['Intersection ID'])].iterrows():
    Const_Loc.loc[j,'ContNum'] = r.ContNum
    Const_Loc.loc[j,'X'] = Int_DF.loc[r['Intersection ID'],'X']
    Const_Loc.loc[j,'Y'] = Int_DF.loc[r['Intersection ID'],'Y']
    j += 1
for i,r in Const_Seg[~pd.isnull(Const_Seg.BEG_STA) & ~pd.isnull(Const_Seg.END_STA) & ~pd.isnull(Const_Seg.INVENTORY)].iterrows():
    Const_Loc.loc[j,'ContNum'] = r.ContNum
    Const_Loc.loc[j,'INVENTORY'] = r.INVENTORY
    Const_Loc.loc[j,'BMP'] = r.BEG_STA
    Const_Loc.loc[j,'EMP'] = r.END_STA
    j += 1

add_cont = list(Const_DC.loc[Const_DC.Location.isin(['added a segment','added segment','Partially OK - 1 segment added','added segments','Added as a segment'])].index.droplevel(1))
for i,r in WPPS_Int[WPPS_Int.ContNum.isin(add_cont)].iterrows():
    Const_Loc.loc[j,'ContNum'] = r.ContNum
    Const_Loc.loc[j,'X'] = r.X
    Const_Loc.loc[j,'Y'] = r.Y
    j += 1
for i,r in WPPS_Seg[WPPS_Seg.ContNum.isin(add_cont)].iterrows():
    Const_Loc.loc[j,'ContNum'] = r.ContNum
    Const_Loc.loc[j,'INVENTORY'] = r.INVENTORY
    Const_Loc.loc[j,'BMP'] = r.BEG_STA
    Const_Loc.loc[j,'EMP'] = r.END_STA
    j += 1
    
Const_Loc.loc[Const_Loc.BMP<0,'BMP'] = 0
Const_Loc

,ContNum,X,Y,INVENTORY,BMP,EMP
0,60A25,2.89189e+06,1.82243e+06,NaN,NaN,NaN
1,60C09,2.92483e+06,1.94952e+06,NaN,NaN,NaN
2,60C14,2.92836e+06,1.97101e+06,NaN,NaN,NaN
3,60C15,2.95059e+06,1.80677e+06,NaN,NaN,NaN
4,60D19,2.92261e+06,1.83631e+06,NaN,NaN,NaN
5,60D20,3.00667e+06,1.75019e+06,NaN,NaN,NaN
6,60D20,3.00645e+06,1.75548e+06,NaN,NaN,NaN
7,60D20,3.00621e+06,1.76004e+06,NaN,NaN,NaN
8,60D20,3.00597e+06,1.76625e+06,NaN,NaN,NaN
9,60D51,2.83225e+06,1.88733e+06,NaN,NaN,NaN


In [2]:
int_df = pd.read_excel(r"\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\11.Plans_DataCollection\PlansDataCollection_Systemic_01112019.xlsx",sheet_name='Intersections')
seg_df = pd.read_excel(r"\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\11.Plans_DataCollection\PlansDataCollection_Systemic_01112019.xlsx",sheet_name='Segments')
print(int_df.shape,seg_df.shape)
int_df.ContNum = int_df.ContNum.astype(str).fillna('')
seg_df.ContNum = seg_df.ContNum.astype(str).fillna('')
int_df.X = int_df.X.astype(float).fillna(0)
int_df.Y = int_df.Y.astype(float).fillna(0)
seg_df.BEG_STA = seg_df.BEG_STA.apply(pd.to_numeric, errors='coerce').fillna(0)
seg_df.END_STA = seg_df.END_STA.apply(pd.to_numeric, errors='coerce').fillna(0)
seg_df.INVENTORY = seg_df.INVENTORY.astype('str').fillna('')
seg_df = seg_df[(seg_df.INVENTORY.apply(len)==17) & (seg_df.BEG_STA<seg_df.END_STA) & (seg_df.ContNum.apply(len)==5)][['ContNum','INVENTORY','BEG_STA','END_STA']]
int_df = int_df[(int_df.X>0) & (int_df.Y>0) & (int_df.ContNum.apply(len)==5)][['ContNum','X','Y']]
print(int_df.shape,seg_df.shape)


(212, 5) (1646, 7)
(202, 3) (1581, 4)


In [8]:
Main_GDB = Project_GDB
Table_FN = Main_GDB + '\\Systemic_Seg_Tab'
SegLayer = Main_GDB + '\\Systemic_Seg_Layer'
SegFC    = Main_GDB + '\\Systemic_Seg'
arcpy.Delete_management(Table_FN)
arcpy.CreateTable_management(out_path=Main_GDB,out_name=os.path.basename(Table_FN))
arcpy.AddField_management(in_table=Table_FN,field_name='INVENTORY',field_type='Text',field_length=17)
arcpy.AddField_management(in_table=Table_FN,field_name='BEG_STA',field_type='Double',field_length=10)
arcpy.AddField_management(in_table=Table_FN,field_name='END_STA',field_type='Double',field_length=10)
arcpy.AddField_management(in_table=Table_FN,field_name='ContNum',field_type='Text',field_length=10)
#arcpy.AddField_management(in_table=Table_FN,field_name='HSIPIDs',field_type='Text',field_length=300)
uc= arcpy.InsertCursor(Table_FN)
for i,r in seg_df.iterrows():
    row = uc.newRow()
    row.setValue('INVENTORY',r.INVENTORY)
    row.setValue('BEG_STA',r.BEG_STA)
    row.setValue('END_STA',r.END_STA)
    row.setValue('ContNum',r.ContNum)
 #   row.setValue('HSIPIDs',r.HSIPIDs)
    uc.insertRow(row)
del uc
arcpy.Delete_management(SegFC)
hsmpy3.network.CreateRouteEventLayer(AttTable=Table_FN,Sites_Routes=IRIS_Route[2017],
                                            BMP='BEG_STA',EMP='END_STA',Fields=['ContNum'],
                                            Output=SegFC,RouteID='INVENTORY')
hsmpy3.common.PrintSummary(SegFC)
IntFC    = Main_GDB + '\\System_Int'
arcpy.Delete_management(IntFC)
arcpy.CreateFeatureclass_management(out_path=Main_GDB,out_name=os.path.basename(IntFC),geometry_type='POINT',spatial_reference=hsmpy3.common.NAD1983IL)
arcpy.AddField_management(in_table=IntFC,field_name='ContNum',field_type='Text',field_length=10)
#arcpy.AddField_management(in_table=Table_FN,field_name='HSIPIDs',field_type='Text',field_length=300)
uc= arcpy.InsertCursor(IntFC)
for i,r in int_df.iterrows():
    row = uc.newRow()
    row.setValue('ContNum',r.ContNum)
    Pt =  arcpy.PointGeometry(arcpy.Point(r.X,r.Y),hsmpy3.common.NAD1983IL)
    row.shape = Pt
    uc.insertRow(row)
del uc
hsmpy3.common.PrintSummary(IntFC)
print()

Type: Polyline
Columns: 8 x Rows: 1556
['BEG_STA', 'ContNum', 'END_STA', 'INVENTORY', 'LOC_ERROR', 'OBJECTID', 'Shape', 'Shape_Length']
Type: Point
Columns: 3 x Rows: 202
['ContNum', 'OBJECTID', 'Shape']



In [88]:
Main_GDB = Project_GDB
Table_FN = Main_GDB + '\\Const_Seg_Tab'
SegLayer = Main_GDB + '\\Const_Seg_Layer'
SegFC    = Main_GDB + '\\Const_Seg'
arcpy.Delete_management(Table_FN)
arcpy.CreateTable_management(out_path=Main_GDB,out_name=os.path.basename(Table_FN))
arcpy.AddField_management(in_table=Table_FN,field_name='INVENTORY',field_type='Text',field_length=17)
arcpy.AddField_management(in_table=Table_FN,field_name='BEG_STA',field_type='Double',field_length=10)
arcpy.AddField_management(in_table=Table_FN,field_name='END_STA',field_type='Double',field_length=10)
arcpy.AddField_management(in_table=Table_FN,field_name='ContNum',field_type='Text',field_length=10)
#arcpy.AddField_management(in_table=Table_FN,field_name='HSIPIDs',field_type='Text',field_length=300)
uc= arcpy.InsertCursor(Table_FN)
for i,r in Const_Loc[~pd.isnull(Const_Loc.INVENTORY)].iterrows():
    row = uc.newRow()
    row.setValue('INVENTORY',r.INVENTORY)
    row.setValue('BEG_STA',r.BMP)
    row.setValue('END_STA',r.EMP)
    row.setValue('ContNum',r.ContNum)
 #   row.setValue('HSIPIDs',r.HSIPIDs)
    uc.insertRow(row)
del uc
arcpy.Delete_management(SegFC)
hsmpy3.network.CreateRouteEventLayer(AttTable=Table_FN,Sites_Routes=IRIS_Route[2016],
                                            BMP='BEG_STA',EMP='END_STA',Fields=['ContNum'],
                                            Output=SegFC,RouteID='INVENTORY')
hsmpy3.common.PrintSummary(SegFC)
IntFC    = Main_GDB + '\\Const_Int'
arcpy.Delete_management(IntFC)
arcpy.CreateFeatureclass_management(out_path=Main_GDB,out_name=os.path.basename(IntFC),geometry_type='POINT',spatial_reference=hsmpy3.common.NAD1983IL)
arcpy.AddField_management(in_table=IntFC,field_name='ContNum',field_type='Text',field_length=10)
#arcpy.AddField_management(in_table=Table_FN,field_name='HSIPIDs',field_type='Text',field_length=300)
uc= arcpy.InsertCursor(IntFC)
for i,r in Const_Loc[~pd.isnull(Const_Loc.X) & ~pd.isnull(Const_Loc.Y)].iterrows():
    row = uc.newRow()
    row.setValue('ContNum',r.ContNum)
    Pt =  arcpy.PointGeometry(arcpy.Point(r.X,r.Y),hsmpy3.common.NAD1983IL)
    row.shape = Pt
    uc.insertRow(row)
del uc
hsmpy3.common.PrintSummary(IntFC)
print()

Type: Polyline
Columns: 8 x Rows: 289
['BEG_STA', 'ContNum', 'END_STA', 'INVENTORY', 'LOC_ERROR', 'OBJECTID', 'Shape', 'Shape_Length']
Type: Point
Columns: 3 x Rows: 233
['ContNum', 'OBJECTID', 'Shape']



In [22]:
for ContNum in Const_DC.index.droplevel(1).tolist()[380:400]:
    print(ContNum)
    display(Const_DC.loc[ContNum])
    display(Const_Int[Const_Int.ContNum==ContNum])
    display(Const_Seg[Const_Seg.ContNum==ContNum])

66720


,ProblemDesc,ImproveDesc,Improvements,HasPlans,Geocoded,Mileage,NumInt,AwardDate,CompDate,BeforePeriod,AfterPeriod,TargetCrashs,CompAmount,HSIPAmount,ServiceLife,Location,Treatment,Unnamed: 19
HSIPID,,,,,,,,,,,,,,,,,,
201410098,Section was a HAL segment and is the only rema...,"Add a Bi-Directional Turn Lane, Modernize sign...",Add Mast Arms and Signal Head per Lane;Other I...,1,1,0.318292,0,2008-10-10,2010-04-30,2005;2006;2007,2011;2012;2013;2014;2015;2016,Angle;RearEnd;Turning,2193533.76,1406096.11,15,OK,"Widening and/or Resurfacing, Traffic Signal Im...",NaN


,ContNum,Intersection ID,Unnamed: 2


,ContNum,INVENTORY,BEG_STA,END_STA,NOTES


66739


...

,ContNum,INVENTORY,BEG_STA,END_STA,NOTES


93468


,ProblemDesc,ImproveDesc,Improvements,HasPlans,Geocoded,Mileage,NumInt,AwardDate,CompDate,BeforePeriod,AfterPeriod,TargetCrashs,CompAmount,HSIPAmount,ServiceLife,Location,Treatment,Unnamed: 19
HSIPID,,,,,,,,,,,,,,,,,,
201410166,"Offset intersection, profile on mainline being...",Realign to remove offset intersection. Raise v...,Improvement/Realignment/Reconstruction;Other I...,1,1,0.010014,0,2008-10-22,2009-08-28,2005;2006;2007,2010;2011;2012;2013;2014;2015;2016,NaN,413233.41,375775.94,15,added segments (original was too short),"Shoulder Improvement/Widening, Improvement/Rea...",NaN


,ContNum,Intersection ID,Unnamed: 2


,ContNum,INVENTORY,BEG_STA,END_STA,NOTES
124,93468,084 70155 000000,1.778,3.787,NaN
125,93468,084 70484 000000,0.000,0.258,NaN


In [30]:
#Const_Seg = hsmpy3.common.FCtoDF(Project_GDB+'\\Const_Seg',readGeometry=True)
#Const_Int = hsmpy3.common.FCtoDF(Project_GDB+'\\Const_Int',readGeometry=True)
sum([s.length/5280 for s in Const_Seg.Shape])
Const_Int.shape[0]

342